In [ ]:
import simulate
import withdrawal
import market
import harvesting
import metrics
import lens

from decimal import Decimal as D
import decimal
import itertools
import pandas
import numpy
import math
#import ipysheet

In [ ]:
%matplotlib inline
import seaborn
from matplotlib import pyplot as plt
import matplotlib
seaborn.set(style="whitegrid")
seaborn.set_context('poster')

In [ ]:
Omegas = [harvesting.make_omeganot(D(n)/100) for n in range(70,140, 10)]
Rebalances = [harvesting.make_rebalancer(D(n)/100) for n in range(20, 110, 10)]
Glidepaths = [harvesting.Glidepath, harvesting.InverseGlidepath, harvesting.AgeBased_100]

Omega100 = harvesting.make_omeganot(D(1))
Rebalance75 = harvesting.make_rebalancer(D(75)/100)

Constant_04 = withdrawal.make_constantdollar(D('.04'))
Constant_05 = withdrawal.make_constantdollar(D('.05'))

In [ ]:
def run2(a_lens, harvest1, harvest2, withdraw=Constant_05, length=40):
    s1 = simulate.calc_lens(harvest1, withdraw, length, a_lens)
    s2 = simulate.calc_lens(harvest2, withdraw, length, a_lens)
    df = pandas.DataFrame({harvest1.__name__: s1, harvest2.__name__: s2})
        
    g = seaborn.relplot(
        data=df.loc[1965:1975],
#        aspect=4,
    )
    g.fig.autofmt_xdate()
    g.despine(left=True, bottom=True, offset=20)
    g.fig.suptitle('Individual year comparisons: 1965 - 1975')
    return (g, df)

def run_deciles(a_lens, harvest1, harvest2, withdraw=Constant_05, length=40):
    s1 = simulate.calc_lens(harvest1, withdraw, length, a_lens).astype(float)
    s2 = simulate.calc_lens(harvest2, withdraw, length, a_lens).astype(float)
    
    bins = 5

    df = pandas.DataFrame(index=numpy.arange(0, 100, bins), columns=[harvest1.__name__, harvest2.__name__])
    for i in range(0, 100, bins):
        df.loc[i] = (s1.quantile(i/100), s2.quantile(i/100))
    
    g = seaborn.relplot(
        data=df,
        aspect=2,
    )
    g.despine(left=True, bottom=True, offset=20)
    g.fig.suptitle('Grouped by quantiles (5%, 10%, 15%, etc.)')    
    return (g, df)

def by_year(year, lens, harvest1, harvest2, years=40):
    MARKET = market.Returns_US_1871()
    s1 = simulate.withdrawals(MARKET.iter_from(year), harvesting=harvest1, withdraw=withdrawal.make_vpw(years+5), years=years)
    s2 = simulate.withdrawals(MARKET.iter_from(year), harvesting=harvest2, withdraw=withdrawal.make_vpw(years+5), years=years)

    df = pandas.DataFrame({harvest1.__name__: [lens(x) for x in s1], harvest2.__name__: [lens(x) for x in s2]})
    g = seaborn.relplot(
        data=df[df[harvest1.__name__] < 40000],
        aspect=4,
    )
    g.despine(left=True, bottom=True, offset=20)
    g.fig.suptitle(year)
    return (g, df)

def ulcer_floor(series, floor=40000):
    sumsq = 0
    for value in [float(n) for n in series]:
        if value < floor:
            drawdown = (100 * ((value / floor) - 1))
            sq = drawdown ** 2
            sumsq += sq
    return math.sqrt(sumsq / len(series)) / 100

def wd_r(x):
    return x.withdraw_r
def port_r(x):
    return x.portfolio_pre.value_r

In [ ]:
L = 40

# Shortfall Years

In [ ]:
run_deciles(lens.calc_shortfall_years, Omega100, Rebalance75)[0]
run2(lens.calc_shortfall_years, Omega100, Rebalance75)[0]

# PWA0

In [ ]:
run_deciles(lens.calc_pwa0, Omega100, Rebalance75)[0]
run2(lens.calc_pwa0, Omega100, Rebalance75)[0]

# Coverage Ratio

In [ ]:
run_deciles(lambda a: lens.calc_coverage_ratio(a, L), Omega100, Rebalance75, length=L)[0]
run2(lambda a: lens.calc_coverage_ratio(a, L), Omega100, Rebalance75, length=L)[0]

# Ulcer Index (Portfolio)

In [ ]:
run_deciles(lens.calc_ulcer, Omega100, Rebalance75)[0]
run2(lens.calc_ulcer, Omega100, Rebalance75)[0]

# Ulcer Index (Withdrawals)

In [ ]:
run_deciles(lambda a: metrics.ulcer([n.withdraw_r for n in a]), Omega100, Rebalance75, withdraw=withdrawal.make_vpw(L+5))[0]
run2(lambda a: metrics.ulcer([n.withdraw_r for n in a]), Omega100, Rebalance75, withdraw=withdrawal.make_vpw(L+5))[0]

# CEW

In [ ]:
run_deciles(lens.calc_cew, Omega100, Rebalance75, withdraw=withdrawal.make_vpw(L+5))[0]
g, df = run2(lens.calc_cew, Omega100, Rebalance75, withdraw=withdrawal.make_vpw(L+5))
g
df.loc[1969]

# HREFF-4

In [ ]:
run_deciles(lens.calc_hreff, Omega100, Rebalance75, withdraw=withdrawal.make_vpw(L+5))[0]
g, df = run2(lens.calc_hreff, Omega100, Rebalance75, withdraw=withdrawal.make_vpw(L+5))
g

In [ ]:
df.loc[1969]

# HREFF-3

In [ ]:
run_deciles(lambda a: lens.calc_hreff(a, floor=D('.025')), Omega100, Rebalance75, withdraw=withdrawal.make_vpw(L+5))[0]
g, df = run2(lambda a: lens.calc_hreff(a, floor=D('.025')), Omega100, Rebalance75, withdraw=withdrawal.make_vpw(L+5))
g

In [ ]:
df.loc[1969]

# Ulcer Floor (Withdrawals)

In [ ]:
run_deciles(lambda a: ulcer_floor([n.withdraw_r for n in a]), Omega100, Rebalance75, withdraw=withdrawal.make_vpw(L+5))[0]
g, df = run2(lambda a: ulcer_floor([n.withdraw_r for n in a]), Omega100, Rebalance75, withdraw=withdrawal.make_vpw(L+5))
g
df.loc[1969]

# Compare Yearly Withdrawals

In [ ]:
for i in range(1928, 1931):
    by_year(i, wd_r, Omega100, Rebalance75)[0]

# Build difference of all withdrawals for all years

We only care about years where the Rebalance strategy fell below our floor of $40,000

In [ ]:
def build_ulcer_diffs(harvest1, harvest2, floor=40000):
    L = 40
    MARKET = market.Returns_US_1871()
    end_year = 2018 - L + 1
    
    ulcers = pandas.Series(index=numpy.arange(MARKET.start_year, end_year))
    
    for start in range(MARKET.start_year, end_year):
        sim1 = simulate.withdrawals(MARKET.iter_from(start), harvesting=harvest1, withdraw=withdrawal.make_vpw(L+5), years=L)
        sim2 = simulate.withdrawals(MARKET.iter_from(start), harvesting=harvest2, withdraw=withdrawal.make_vpw(L+5), years=L)

        s1 = pandas.Series([wd_r(x) for x in sim1]).clip(upper=floor)
        s2 = pandas.Series([wd_r(x) for x in sim2]).clip(upper=floor)
        ulcers.loc[start] = metrics.ulcer(s2) - metrics.ulcer(s1)

    return ulcers

# Build Ulcer Diff for all years

A higher number means Rebalance was *worse*

In [ ]:
ulcer_diffs = build_ulcer_diffs(Omega100, Rebalance75)
ulcer_diffs.sort_values(ascending=False).head()

In [ ]:
for x in ulcer_diffs.sort_values(ascending=False).index[0:5]:
    by_year(x, wd_r, Omega100, Rebalance75)[0]

# The opposite

But we also want to check the opposite...cases where OmegaNot100 does *worse*

In [ ]:
ulcer_diffs.sort_values(ascending=True).head()

In [ ]:
for x in ulcer_diffs.sort_values(ascending=True).index[0:5]:
    by_year(x, wd_r, Omega100, Rebalance75)[0]